In [1]:
from clients import MysqlClient

In [2]:
import os
os.environ['RDS_MYSQL_PW'] = ''
os.environ['AWS_ACCESS_KEY_ID'] = ''
os.environ['AWS_SECRET_ACCESS_KEY'] = ''
os.environ['AWS_REGION_NAME'] = "ap-northeast-2"

In [3]:
mysql = MysqlClient()

# selenium

In [6]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import NoSuchElementException
import time
import pandas as pd
import datetime as dt


In [43]:
driver = webdriver.Chrome()

In [65]:
daum_movies = mysql.get_daum_movies()
daum_movies

/Users/kyeongchanlee/PycharmProjects/recsys_service_deployment/clients.py:37: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(sql='select * from daum_movies', con=connection)


,movieId,titleKo,titleEn,mainPageUrl,posterUrl
0,261,월드 워 Z,None,https://movie.daum.net/moviedb/main?movieId=48708,https://img1.daumcdn.net/thumb/C408x596/?fname...
1,262,은밀하게 위대하게,None,https://movie.daum.net/moviedb/main?movieId=69884,https://img1.daumcdn.net/thumb/C408x596/?fname...
2,263,맨 오브 스틸,None,https://movie.daum.net/moviedb/main?movieId=43811,https://img1.daumcdn.net/thumb/C408x596/?fname...
3,264,스타트렉 다크니스,None,https://movie.daum.net/moviedb/main?movieId=62929,https://img1.daumcdn.net/thumb/C408x596/?fname...
4,265,더 콜,None,https://movie.daum.net/moviedb/main?movieId=71178,https://img1.daumcdn.net/thumb/C408x596/?fname...
...,...,...,...,...,...
2872,3133,세븐데이즈,None,https://movie.daum.net/moviedb/main?movieId=43417,None
2873,3134,샤이엔 워리어,None,https://movie.daum.net/moviedb/main?movieId=14...,None
2874,3135,자귀모,None,https://movie.daum.net/moviedb/main?movieId=10,None
2875,3136,레드 던,None,https://movie.daum.net/moviedb/main?movieId=56027,None


# 포스터만 따로 수집할것들

In [63]:
import pandas as pd
with mysql.get_connection() as connection:
    daum_ratings = pd.read_sql(sql='select * from daum_ratings', con=connection)
daum_ratings

/var/folders/cg/2vt5wbq5767fsxv0005k5x4m0000gn/T/ipykernel_5545/3919600470.py:3: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  daum_ratings = pd.read_sql(sql='select * from daum_ratings', con=connection)


,nickName,movieId,rating,timestamp,userId,review
0,닉네임,2880,10,1.700658e+09,None,좀비영화 싫어하는데 이건 넘 재밌네
1,닉네임,2881,8,1.668779e+09,None,의상 너무너무 예쁘다
2,닉네임,2564,9,1.657807e+09,None,오프닝 너~~~무 조으다\n별5개 다 못주는건 제니퍼..왜 나오셨쎄요?ㅜㅜ
3,닉네임,2351,5,1.648305e+09,None,햄릿이 넘 왕자같지않아. 몰입이 안돼
4,닉네임,2357,9,1.627718e+09,None,끝장면 과하지않아 넘넘좋다\n코끝이 찡~잘 만들어진 영화\n만점을 못준건 어떤 배우...
...,...,...,...,...,...,...
576,눈이부시게 그리움,412,9,1.504651e+09,None,재밌으면 되잖여ㅋㅋ
577,눈이부시게 그리움,1206,5,1.487001e+09,None,그럼에도 불구하고 일본은 아직도 기독교가 뿌리를 못내림\n태생이 진짜 악마스러워\n...
578,눈이부시게 그리움,2889,10,1.484782e+09,None,몇년만에 다시봐도 엄지척을 안할수가 없네요\n장국영의 연기는 최고
579,눈이부시게 그리움,1069,10,1.475426e+09,None,역시 송강호...공유도 멋져멋져..잘 봤어요


In [55]:
daum_movies[daum_movies['titleKo'] == '과속스캔들']

,movieId,titleKo,titleEn,mainPageUrl,posterUrl
2728,2989,과속스캔들,None,https://movie.daum.net/moviedb/main?movieId=48057,None


In [50]:
sum(daum_movies.posterUrl.notnull())

2618

In [45]:
# title_ko_lst = []
# mainpageurl_lst = []
sunday_dt = dt.datetime(2013, 6, 23)
select_query = "SELECT * FROM daum_movies WHERE titleKo = %s and mainPageUrl = %s"  # 데이터 있는지 확인
insert_query = "INSERT INTO daum_movies (titleKo, mainPageUrl, posterUrl) VALUES (%s, %s, %s)"  # 삽입 쿼리

for _ in range(2000):
    driver.get(f"https://movie.daum.net/ranking/boxoffice/weekly?date={sunday_dt.strftime('%Y%m%d')}")
    week_movies_lst = driver.find_elements(By.CSS_SELECTOR, '#mainContent > div > div.box_boxoffice > ol > li')

    for movie in week_movies_lst:
        try:
            a_title = movie.find_element(By.CSS_SELECTOR, 'div > div.thumb_cont > strong > a')
            mainpageurl = a_title.get_attribute('href')
            title_ko = a_title.get_attribute('text')
            poster_url = movie.find_element(By.CSS_SELECTOR, 'div.poster_movie img').get_attribute('src')

            # 바로 테이블로 삽입 ################
            # title_ko, mainpageurl 쌍이 존재하는지 확인
            with mysql.get_connection().cursor() as cursor:
                cursor.execute(select_query, (title_ko, mainpageurl))
                exist = cursor.fetchall()
            # 없으면 삽입
            if not exist:
                data_to_insert = (title_ko, mainpageurl, poster_url)
                with mysql.get_connection() as conn:
                    cursor = conn.cursor()
                    cursor.execute(insert_query, data_to_insert)
                    conn.commit()
            ################################
        except NoSuchElementException:
            pass
    if sunday_dt.month in [6, 12]:
        print(sunday_dt)
    
    sunday_dt += dt.timedelta(days=7)


2013-06-23 00:00:00
2013-06-30 00:00:00
2013-12-01 00:00:00
2013-12-08 00:00:00
2013-12-15 00:00:00
2013-12-22 00:00:00
2013-12-29 00:00:00
2014-06-01 00:00:00
2014-06-08 00:00:00
2014-06-15 00:00:00
2014-06-22 00:00:00
2014-06-29 00:00:00
2014-12-07 00:00:00
2014-12-14 00:00:00
2014-12-21 00:00:00
2014-12-28 00:00:00
2015-06-07 00:00:00
2015-06-14 00:00:00
2015-06-21 00:00:00
2015-06-28 00:00:00
2015-12-06 00:00:00
2015-12-13 00:00:00
2015-12-20 00:00:00
2015-12-27 00:00:00
2016-06-05 00:00:00
2016-06-12 00:00:00
2016-06-19 00:00:00
2016-06-26 00:00:00
2016-12-04 00:00:00
2016-12-11 00:00:00
2016-12-18 00:00:00
2016-12-25 00:00:00
2017-06-04 00:00:00
2017-06-11 00:00:00
2017-06-18 00:00:00
2017-06-25 00:00:00
2017-12-03 00:00:00
2017-12-10 00:00:00
2017-12-17 00:00:00
2017-12-24 00:00:00
2017-12-31 00:00:00
2018-06-03 00:00:00
2018-06-10 00:00:00
2018-06-17 00:00:00
2018-06-24 00:00:00
2018-12-02 00:00:00
2018-12-09 00:00:00
2018-12-16 00:00:00
2018-12-23 00:00:00
2018-12-30 00:00:00


KeyboardInterrupt: 

In [227]:
daum_movies = mysql.get_daum_movies()
daum_movies

/Users/kyeongchanlee/PycharmProjects/recsys_service_deployment/clients.py:37: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(sql='select * from daum_movies', con=connection)


,movieId,titleKo,titleEn,mainPageUrl,posterUrl
0,261,월드 워 Z,None,https://movie.daum.net/moviedb/main?movieId=48708,https://img1.daumcdn.net/thumb/C408x596/?fname...
1,262,은밀하게 위대하게,None,https://movie.daum.net/moviedb/main?movieId=69884,https://img1.daumcdn.net/thumb/C408x596/?fname...
2,263,맨 오브 스틸,None,https://movie.daum.net/moviedb/main?movieId=43811,https://img1.daumcdn.net/thumb/C408x596/?fname...
3,264,스타트렉 다크니스,None,https://movie.daum.net/moviedb/main?movieId=62929,https://img1.daumcdn.net/thumb/C408x596/?fname...
4,265,더 콜,None,https://movie.daum.net/moviedb/main?movieId=71178,https://img1.daumcdn.net/thumb/C408x596/?fname...
...,...,...,...,...,...
7266,7527,보더타운,None,https://movie.daum.net/moviedb/main?movieId=43069,None
7267,7528,베테랑,None,https://movie.daum.net/moviedb/main?movieId=64524,None
7268,7529,감각의 제국,None,https://movie.daum.net/moviedb/main?movieId=133,None
7269,7530,오발탄,None,https://movie.daum.net/moviedb/main?movieId=1628,None


In [228]:
daum_ratings = mysql.get_daum_ratings()
daum_ratings

/Users/kyeongchanlee/PycharmProjects/recsys_service_deployment/clients.py:42: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(sql='select * from daum_ratings', con=connection)


,nickName,movieId,rating,timestamp,userId,review
0,닉네임,2880,10,1.700658e+09,None,좀비영화 싫어하는데 이건 넘 재밌네
1,닉네임,2881,8,1.668779e+09,None,의상 너무너무 예쁘다
2,닉네임,2564,9,1.657807e+09,None,오프닝 너~~~무 조으다\n별5개 다 못주는건 제니퍼..왜 나오셨쎄요?ㅜㅜ
3,닉네임,2351,5,1.648305e+09,None,햄릿이 넘 왕자같지않아. 몰입이 안돼
4,닉네임,2357,9,1.627718e+09,None,끝장면 과하지않아 넘넘좋다\n코끝이 찡~잘 만들어진 영화\n만점을 못준건 어떤 배우...
...,...,...,...,...,...,...
20128,운동해야지,2081,0,1.595246e+09,None,연상호는 그저 운이 좋았을뿐 전작도 신파에 그저 그랬는데 과대평가됐지
20129,운동해야지,3247,9,1.584676e+09,None,"지져스, 이래서 헐리웃 작가들이 천문학적 돈을 받는 이유가 있는 거지, 얼마나 고증..."
20130,운동해야지,4357,5,1.582536e+09,None,근데 이 집의 가족 구성이 어찌 된 일이에요? 아빠는 백인 엄마가 코리언이고 애들은...
20131,운동해야지,1962,1,1.579430e+09,None,한국 영화계갖어쩌다 이렇게 됐을꼬. 보통 좌익정권하에 좋은 영화들이 많이 탄생되는데...


In [225]:
daum_ratings[daum_ratings['nickName'] == '릴리 슈슈']

,nickName,movieId,rating,timestamp,userId,review
19458,릴리 슈슈,2857,10,1.699335e+09,None,엄마없이 사는건 이렇게나 큰 충격인거다\nㅠㅠ
19459,릴리 슈슈,2694,10,1.697943e+09,None,왓챠 만료일에 마음먹고 본 영화\n위로와 사랑이 필요한 나에게 딱
19460,릴리 슈슈,2780,10,1.688710e+09,None,추억으로 가는 길
19461,릴리 슈슈,7423,10,1.686323e+09,None,지금 kbs 독립영화관에서 봤어요
19462,릴리 슈슈,7424,10,1.685287e+09,None,한숨이 나오지만 진짜인 영화
19463,릴리 슈슈,729,10,1.684808e+09,None,아버지도 아들도 이제 안계시다네요...\n하늘에서 더 행복하시길...
19464,릴리 슈슈,2612,10,1.683972e+09,None,갓투리\n너무 감동적 ㅠㅠ
19465,릴리 슈슈,3631,10,1.682934e+09,None,진짜 너무 깜짝놀람\n뭐 하나 완벽하지 않은게 없네요
19466,릴리 슈슈,4849,10,1.682934e+09,None,황홀경\n\n이제라도 이 영화를 본 나를 칭찬하겠습니다
19467,릴리 슈슈,7425,10,1.678764e+09,None,브렌든 프레이저 키 후이 콴이 오스카를 나란히 받을 줄은 상상도 못함 ㅠ♡\n암...


In [133]:
mysql.get_count(table_name='daum_ratings')

12315

In [119]:
def get_title_ko(movie_id):
    with mysql.get_connection().cursor() as cursor:
        cursor.execute(f"select titleKo from daum_movies where movieId = '{movie_id}'")
        result = cursor.fetchall()[0][0]
    return result

In [120]:
get_title_ko(265)

'더 콜'

In [103]:
with mysql.get_connection().cursor() as cursor:
    cursor.execute('select titleKo from daum_movies where movieId in (278, 5735, 2880, 262, 265)')
    result = cursor.fetchall()
result

(('은밀하게 위대하게',), ('더 콜',), ('빅 웨딩',), ('월드워Z',), ('사랑은 타이핑 중!',))

In [226]:
nickname = '릴리 슈슈'
with mysql.get_connection().cursor() as cursor:
    cursor.execute(f"select movieId, timestamp, rating, review from daum_ratings where nickName = '{nickname}'")
    result = cursor.fetchall()

for box in sorted(list(result), key=lambda x: x[1]):
    movie_id, ts, rating, review = box
    review = review.replace('\n', ' ')
    print(f"{dt.datetime.fromtimestamp(ts)}\t{rating}\t[{get_title_ko(movie_id)}] - {review}")
    

2019-11-07 13:57:00	10	[비포 미드나잇] - before the dawn....기다릴게요.
2022-02-19 14:29:00	10	[로마의 휴일] - 볼 때마다 우는데 저만 그래요?
2022-03-07 13:41:00	10	[피그] - 허기진 마음이 채워짐
2022-04-29 04:07:00	10	[일본곤충기] - 마츠코 전에 마츠키가 있었네
2022-05-10 23:39:00	10	[하이앤로우 더 무비] - 잘생긴 애들이 폼 잡는 것도 너무 귀엽고  무엇보다 음악이 !!!
2022-05-31 14:24:00	10	[범죄도시2] - 나쁜놈은 잡아야 맛이지
2022-06-19 00:42:00	10	[브로커] - 해진이한테 만 상주고싶다
2022-06-28 13:36:00	10	[탑건: 매버릭] - 끝내준다! 탐 크루즈 건강하세요!♡
2022-06-30 00:30:00	10	[탑건] - 톰 크루즈 진짜 너무귀엽다 ㅠ
2022-12-07 20:08:00	10	[사랑의 하모니] - 저도  초등학교때 ㅋ 대사를 외우도록 본 영화네요. 유튜브에 풀버전 있어요.
2022-12-29 14:17:00	10	[아바타: 물의 길] - 아빠생각만 나게하고 ㅠ 너무해 ㅠ♡
2023-03-01 02:32:00	10	[더 웨일] - 내 첫사랑  원시 틴에이저♡ 응원합니다
2023-03-06 10:26:00	10	[TAR 타르] - 번스타인의 말러가 듣고싶어졌다
2023-03-14 12:15:00	10	[원시 틴에이저] - 브렌든 프레이저  키 후이 콴이 오스카를 나란히 받을 줄은 상상도 못함  ㅠ♡ 암튼 영화 재밌는데 정말 다시보고싶네요
2023-05-01 18:39:00	10	[2001: 스페이스 오디세이] - 황홀경  이제라도 이 영화를 본 나를 칭찬하겠습니다
2023-05-01 18:46:00	10	[블레이드 러너: 파이널 컷] - 진짜 너무 깜짝놀람 뭐 하나 완벽하지 않은게 없네요
2023-05-13 19:00:00	10	[극장판 엄마 까투리: 도시로 간 까투

In [124]:
movie_id, ts, rating, review = box

In [125]:
get_title_ko(movie_id)

'마마'

In [128]:
print(dt.datetime.fromtimestamp(ts))

2013-01-25 11:40:00


In [87]:
from collections import Counter

In [ ]:
daum_ratings

In [89]:
Counter(daum_ratings['movieId'])

Counter({265: 103,
         261: 96,
         264: 69,
         263: 63,
         2880: 48,
         262: 39,
         1797: 32,
         1770: 29,
         1882: 28,
         2081: 26,
         2763: 25,
         1031: 25,
         2564: 24,
         2592: 24,
         1945: 23,
         2544: 23,
         2805: 23,
         2092: 23,
         2357: 21,
         1703: 21,
         2785: 21,
         2595: 19,
         2455: 19,
         2388: 19,
         2424: 19,
         1840: 18,
         2589: 17,
         1792: 17,
         1679: 16,
         3173: 16,
         1502: 16,
         1967: 16,
         1639: 15,
         537: 15,
         2690: 15,
         2611: 15,
         1451: 15,
         3164: 15,
         2540: 15,
         2412: 15,
         389: 15,
         2566: 15,
         609: 14,
         2572: 14,
         1419: 14,
         2675: 14,
         787: 14,
         2193: 14,
         1732: 14,
         1824: 14,
         2893: 13,
         865: 13,
         2780: 13,
  

In [ ]:
2592

In [34]:
insert_query = "INSERT INTO daum_movies (movieId, titleKo, mainPageUrl) VALUES (%s, %s, %s)"
# 삽입할 데이터 정의
data_to_insert = (0, title_ko, mainpageurl)

In [35]:
data_to_insert

(0, '루니 툰 : 백 인 액션', 'https://movie.daum.net/moviedb/main?movieId=38865')

In [37]:
# 쿼리 실행

with mysql.get_connection() as conn:
    cursor = conn.cursor()
    cursor.execute(insert_query, data_to_insert)
    conn.commit()

In [41]:
existing_data

((0,
  '루니 툰 : 백 인 액션',
  None,
  'https://movie.daum.net/moviedb/main?movieId=38865',
  None),)

In [59]:
import pickle

In [62]:
len(set(title_ko_lst))

4405

In [64]:
with open('title_ko_lst.pkl', 'wb') as f:
    pickle.dump(title_ko_lst, f)

In [65]:
with open('mainpageurl_lst.pkl', 'wb') as f:
    pickle.dump(mainpageurl_lst, f)

In [56]:

#     {
#     'title_ko' : title_ko,
#     'mainpageurl' : mainpageurl
# }

In [1]:
title_ko_lst

NameError: name 'title_ko_lst' is not defined

In [2]:
import pickle

In [6]:
with open('title_ko_lst.pkl', 'rb') as f:
    title_ko_lst = pickle.load(f)
with open('mainpageurl_lst.pkl', 'rb') as f:
    mainpageurl_lst = pickle.load(f)

# 바로 테이블로 삽입 

In [5]:
movies

,movieId,title,genres,url
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,https://m.media-amazon.com/images/M/MV5BMDU2ZW...
1,2,Jumanji (1995),Adventure|Children|Fantasy,https://m.media-amazon.com/images/M/MV5BZTk2Zm...
2,3,Grumpier Old Men (1995),Comedy|Romance,https://m.media-amazon.com/images/M/MV5BMjQxM2...
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance,https://m.media-amazon.com/images/M/MV5BYzcyMD...
4,5,Father of the Bride Part II (1995),Comedy,https://m.media-amazon.com/images/M/MV5BOTEyNz...
...,...,...,...,...
62418,209157,We (2018),Drama,None
62419,209159,Window of the Soul (2001),Documentary,None
62420,209163,Bad Poems (2018),Comedy|Drama,None
62421,209169,A Girl Thing (2001),(no genres listed),None


In [6]:
mysql.get_table_names()

Tables : 
daum_movies
movies
ratings


In [9]:
mysql.get_data_type('movies')

Column: movieId, Data Type: int
Column: title, Data Type: varchar(100)
Column: genres, Data Type: varchar(100)
Column: url, Data Type: text


In [7]:
mysql.get_daum_movies()

/Users/kyeongchanlee/PycharmProjects/recsys_service_deployment/clients.py:37: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(sql='select * from daum_movies', con=connection)


,movieId,titleKo,titleEn,mainPageUrl,posterUrl


In [9]:
import pickle
with open('title_ko_lst.pkl', 'rb') as f:
    title_ko_lst = pickle.load(f)
with open('mainpageurl_lst.pkl', 'rb') as f:
    mainpageurl_lst = pickle.load(f)

In [11]:
len(title_ko_lst)

14985

In [12]:
len(mainpageurl_lst)

14985

In [20]:
with mysql.get_connection().cursor() as cursor:
    cursor.execute("SELECT MAX(movieId) FROM daum_movies")
    max_value = cursor.fetchone()[0]

In [21]:
max_value = 0 if not max_value else cursor.fetchone()[0]

In [22]:
max_value

0

In [19]:
int(max_value)

TypeError: int() argument must be a string, a bytes-like object or a real number, not 'NoneType'

In [18]:
int(max_value) + 1

TypeError: int() argument must be a string, a bytes-like object or a real number, not 'NoneType'

In [23]:
for tko, mpurl in zip(title_ko_lst, mainpageurl_lst):
    with mysql.get_connection().cursor() as cursor:
        cursor.execute("SELECT MAX(movieId) FROM daum_movies")
        max_mid = cursor.fetchone()[0]
        max_mid = 0 if not max_mid else cursor.fetchone()[0]
        curr_mid = max_mid + 1
    break
    

    

In [24]:
tko

'실미도'

In [25]:
mpurl

'https://movie.daum.net/moviedb/main?movieId=34452'

In [14]:
mysql.get_count('daum_movies')

0

In [25]:
with mysql.get_connection().cursor() as cursor:
    cursor.execute("SHOW COLUMNS FROM movies")
    columns = cursor.fetchall()

for column in columns:
    column_name = column[0]
    data_type = column[1]
    print(f"Column: {column_name}, Data Type: {data_type}")


Column: movieId, Data Type: int
Column: title, Data Type: varchar(100)
Column: genres, Data Type: varchar(100)
Column: url, Data Type: text


In [23]:
insert_query = "INSERT INTO daum_movies (column1, column2, column3) VALUES (%s, %s, %s)"

# 삽입할 데이터 정의
data_to_insert = ("value1", "value2", "value3")

# 쿼리 실행
cursor.execute(insert_query, data_to_insert)

# 변경사항 커밋
connection.commit()

4

In [14]:
table_create_sql = '''CREATE TABLE daum_movies (
                                            movieId INT NOT NULL,
                                            titleKo VARCHAR(100),
                                            titleEn VARCHAR(100),
                                            mainPageUrl TEXT,
                                            posterUrl TEXT,
                                            PRIMARY KEY(movieId)
                                            )
                                            '''

In [17]:
mysql.get_table_names()

Tables : 
daum_movies
movies
ratings


In [ ]:
movie_id, title_ko, title_en, poster_url

In [ ]:
with mysql.get_connection()

In [53]:
title_ko

'실미도'

In [51]:
pd.DataFrame({
    'title_ko' : title_ko,
    'mainpageurl' : mainpageurl
})

ValueError: If using all scalar values, you must pass an index

In [43]:
movie

<selenium.webdriver.remote.webelement.WebElement (session="beaa1e5540e9447ecab7ea52ae849ffc", element="9F133918A08B6B00E60D04126BC44D78_element_1492")>

In [41]:
movies

""


In [44]:
a_title = movie.find_element(By.CSS_SELECTOR, 'div > div.thumb_cont > strong > a')

In [47]:
mainpage = a_title.get_attribute('href')

In [48]:
title_ko = a_title.get_attribute('text')

In [4]:
driver.get('https://movie.daum.net/ranking/boxoffice/weekly?date=20040101')

In [ ]:
# 일단 영화 url 다 받아오기. 영화제목, 영화 메인페이지

In [10]:
week_movies_lst = driver.find_elements(By.CSS_SELECTOR, '#mainContent > div > div.box_boxoffice > ol > li')

In [15]:
title = week_movies_lst[0].find_element(By.CSS_SELECTOR, 'div > div.thumb_cont > strong > a')

In [16]:
title.get_attribute('href')

'https://movie.daum.net/moviedb/main?movieId=34452'

In [17]:
title.get_attribute('text')

'실미도'

In [ ]:
#mainContent > div > div.box_boxoffice > ol > li:nth-child(9) > div > div.thumb_cont > strong > a

# beautifulsoup

In [1]:
from bs4 import BeautifulSoup
from urllib.robotparser import RobotFileParser
from requests.compat import urlparse, urljoin
from urllib.request import urlopen, Request
import requests
from requests.exceptions import HTTPError
import time
import re

In [2]:
def canfetch(url, agent='*', path='/'):
    robot = RobotFileParser(urljoin(url, '/robots.txt'))
    robot.read()
    return robot.can_fetch(agent, urlparse(url)[2])


def download(url, params={}, headers={}, method='GET', limit=3):
    if canfetch(url) is False:
        print('[Error] ' + url)
#     else:
    try:
        resp = requests.request(method,
                                url,
                                params=params if method == 'GET' else {},
                                data=params if method == 'POST' else {},
                                headers=headers)
        resp.raise_for_status()
    except HTTPError as e:
        if limit > 0 and e.response.status_code >= 500:
            print(limit)
            time.sleep(1)  # => random
            resp = download(url, params, headers, method, limit-1)
        else:
            print('[{}] '.format(e.response.status_code) + url)
            print(e.response.status_code)
            print(e.response.reason)
            print(e.response.headers)
    return resp

In [5]:
url = 'https://movie.daum.net/ranking/boxoffice/yearly?date=2021'
resp = download(url)
dom = BeautifulSoup(resp.content, 'html.parser')

In [6]:
atag.name

NameError: name 'atag' is not defined

In [37]:
atag.attrs


{'href': '/contents/news',
 'class': ['link_txt'],
 'data-tiara-layer': 'contents'}

In [34]:
atag['href']

'/contents/news'

In [8]:
url = 'https://movie.daum.net/moviedb/grade?movieId=122749'
resp = download(url)
dom = BeautifulSoup(resp.content, 'html.parser')

In [9]:
dom

<!DOCTYPE html>

<html lang="ko">
<head>
<meta charset="utf-8"/>
<title>범죄도시2 | 다음영화</title>
<link href="//m2.daumcdn.net/img-media/2010ci/Daum_favicon.ico" rel="shortcut icon"/>
<link href="//t1.daumcdn.net/media/kraken/movie/8c9a93d/PcCommonCssBundle.merged.css" rel="stylesheet" type="text/css">
<script src="//t1.daumcdn.net/media/kraken/movie/8c9a93d/common.merged.js"></script>
<script src="//t1.daumcdn.net/media/kraken/movie/8c9a93d/PcCommonScriptBundle.merged.js"></script>
<script src="//t1.daumcdn.net/cssjs/movie/v2/amchart/amcharts.js"></script>
<script src="//t1.daumcdn.net/cssjs/movie/v2/amchart/serial.js"></script>
<meta content="다음영화" property="og:author">
<meta content="다음영화" property="og:site_name"/>
<meta content="범죄도시2" property="og:title"/>
<meta content="https://t1.daumcdn.net/movie/3f7e3035989e7918c6426e541c864d0d8a295de2" property="og:image"/>
<meta content="Daum영화에서 자세한 내용을 확인하세요!" property="og:description"/>
</meta></link></head>
<body class="">
<div class="direct-

In [7]:
for a in dom.select('a.link_txt'):
    print(f"{a.text} : {a['href']}")
    

스파이더맨: 노 웨이 홈 : /moviedb/main?movieId=146656
모가디슈 : /moviedb/main?movieId=132564
이터널스 : /moviedb/main?movieId=130908
블랙 위도우 : /moviedb/main?movieId=123215
분노의 질주: 더 얼티메이트 : /moviedb/main?movieId=129101
싱크홀 : /moviedb/main?movieId=134290
극장판 귀멸의 칼날: 무한열차편 : /moviedb/main?movieId=145371
베놈 2: 렛 데어 비 카니지 : /moviedb/main?movieId=129680
소울 : /moviedb/main?movieId=136715
크루엘라 : /moviedb/main?movieId=108699
샹치와 텐 링즈의 전설 : /moviedb/main?movieId=129706
인질 : /moviedb/main?movieId=127738
듄 : /moviedb/main?movieId=130886
보이스 : /moviedb/main?movieId=137628
007 노 타임 투 다이 : /moviedb/main?movieId=119916
미나리 : /moviedb/main?movieId=137317
발신제한 : /moviedb/main?movieId=138391
보스 베이비 2 : /moviedb/main?movieId=143722
콰이어트 플레이스 2 : /moviedb/main?movieId=129982
랑종 : /moviedb/main?movieId=144859
유체이탈자 : /moviedb/main?movieId=124768
컨저링3: 악마가 시켰다 : /moviedb/main?movieId=130902
기적 : /moviedb/main?movieId=143697
고질라 VS. 콩 : /moviedb/main?movieId=128987
킹스맨: 퍼스트 에이전트 : /moviedb/main?movieId=125382
엔칸토: 마법의 세계 : /